In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
keep_prob = 0.7

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [9]:
## nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
selu = torch.nn.SELU()

In [10]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0899,  0.0344,  0.0324,  ..., -0.1041, -0.0016,  0.0298],
        [-0.0162, -0.0204, -0.0906,  ..., -0.0675,  0.0825, -0.0610],
        [ 0.0777, -0.0388, -0.0624,  ..., -0.0477,  0.0532, -0.0463],
        ...,
        [ 0.0552, -0.0701, -0.0213,  ..., -0.0114,  0.0745, -0.0336],
        [ 0.0348,  0.0528, -0.1002,  ..., -0.0134, -0.0921,  0.0499],
        [ 0.0743, -0.0307,  0.0216,  ...,  0.0516, -0.0265,  0.0195]],
       requires_grad=True)

In [11]:
# model
model = torch.nn.Sequential(linear1, selu,
                            linear2, selu, 
                            linear3, selu, 
                            linear4, selu, 
                            linear5).to(device)

In [12]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
total_batch = len(data_loader)
model.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.316184044
Epoch: 0002 cost = 0.168593451
Epoch: 0003 cost = 0.138264418
Epoch: 0004 cost = 0.114301436
Epoch: 0005 cost = 0.107449353
Epoch: 0006 cost = 0.085114419
Epoch: 0007 cost = 0.085552946
Epoch: 0008 cost = 0.081255838
Epoch: 0009 cost = 0.078728452
Epoch: 0010 cost = 0.072334081
Epoch: 0011 cost = 0.064717963
Epoch: 0012 cost = 0.065615036
Epoch: 0013 cost = 0.056208249
Epoch: 0014 cost = 0.056877337
Epoch: 0015 cost = 0.056868952
Learning finished


In [14]:
# Test the model using test sets
with torch.no_grad():
    model.eval()
    
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ' , torch.argmax(single_prediction, 1).item())

Accuracy: 0.8751000165939331
Label:  8
Prediction:  8
